# 1. Prerequisite  

In [1]:
!pip install pandas
!pip install numpy
!pip install lightgbm

# 2. Import library

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb

import pickle
import gc
import logging
import os

In [3]:
def Log(file, format, text, level):
    infoLog = logging.FileHandler(file)
    infoLog.setFormatter(format)
    logger = logging.getLogger(file)
    logger.setLevel(level)
    
    if not logger.handlers:
        logger.addHandler(infoLog)
        if (level == logging.INFO):
            logger.info(text)
        if (level == logging.ERROR):
            logger.error(text)
        if (level == logging.WARNING):
            logger.warning(text)
    
    infoLog.close()
    logger.removeHandler(infoLog)
    
    return

formatLOG = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
Log("file.log", formatLOG , " -- Start New Session --", logging.INFO)

#  3. Import dataset

In [4]:
#Import dataset
Log("file.log", formatLOG , "Start importing dataset.", logging.INFO)

# Change path directory accordingly
#Take a smaller sample to speed up test time
train_dataset = pd.read_csv("Data/Processed/dataset_train.csv")
train_dataset = train_dataset.sample(frac=0.1, random_state=1)

# test_dataset = pd.read_csv("Data/Processed/dataset_test.csv")
# test_dataset = test_dataset.sample(frac=0.1, random_state=1)

In [5]:
Log("file.log", formatLOG , "Import dataset Successfully", logging.INFO)

# 4. Cleaning of data

## 4.1 Cleaning Train dataset

In [6]:
Log("file.log", formatLOG , "Cleaning dataset.", logging.INFO)

In [7]:
#Train
train_dataset['timestamp'] = pd.to_datetime(train_dataset['timestamp']) #Convert timestamp to datatime
train_dataset = train_dataset.sort_values(by=['site_id', 'timestamp']) #short values by site id then timestamp
train_dataset.fillna(method = 'ffill', inplace=True, limit = 24)#forward fill the missing data up to 12 hours
train_dataset.fillna(method = 'bfill', inplace=True, limit = 24)#backfill up to 12 hours

#fill NaN cells, set all NaN floor_count to 1 and year_built using mean but i think its not too important at the moment
train_dataset.fillna({'floor_count':1, 'year_built': int(train_dataset['year_built'].mean())}, inplace=True)

train_dataset = train_dataset.sort_values(by=['site_id', 'timestamp']) #short values by site id then timestamp

train_dataset.fillna(method = 'ffill', inplace=True, limit = 24)#forward fill the missing data up to 12 hours
train_dataset.fillna(method = 'bfill', inplace=True, limit = 24)#backfill up to 12 hours

#Get columns with empty cells, subsequently get mean value based on site_id and populate cell
missing_cols = [col for col in train_dataset.columns if train_dataset[col].isna().any()] 
mean_data_by_site_id = train_dataset.groupby('site_id')[missing_cols].transform('mean')
train_dataset.fillna(mean_data_by_site_id, inplace=True)

#Add hour, time of year, and weekend columns
train_dataset['hour'] = train_dataset['timestamp'].dt.hour
train_dataset['day_of_year'] = (train_dataset['timestamp'] - pd.Timestamp('2016-01-01')).dt.days%365
train_dataset['is_weekend'] = train_dataset['timestamp'].dt.weekday.isin([5,6]).astype(int)

In [8]:
pickle.dump( train_dataset, open( "train_dataset.p", "wb" ) )
del train_dataset
gc.collect()

28

## 4.2 Cleaning Test dataset 

In [9]:
# #Test
# test_dataset['timestamp'] = pd.to_datetime(test_dataset['timestamp']) #Convert timestamp to datatime
# test_dataset = test_dataset.sort_values(by=['site_id', 'timestamp']) #short values by site id then timestamp
# test_dataset.fillna(method = 'ffill', inplace=True, limit = 24)#forward fill the missing data up to 12 hours
# test_dataset.fillna(method = 'bfill', inplace=True, limit = 24)#backfill up to 12 hours

# #fill NaN cells, set all NaN floor_count to 1 and year_built using mean but i think its not too important at the moment
# test_dataset.fillna({'floor_count':1, 'year_built': int(test_dataset['year_built'].mean())}, inplace=True)

# test_dataset = test_dataset.sort_values(by=['site_id', 'timestamp']) #short values by site id then timestamp

# test_dataset.fillna(method = 'ffill', inplace=True, limit = 24)#forward fill the missing data up to 12 hours
# test_dataset.fillna(method = 'bfill', inplace=True, limit = 24)#backfill up to 12 hours

# #Get columns with empty cells, subsequently get mean value based on site_id and populate cell
# missing_cols = [col for col in test_dataset.columns if test_dataset[col].isna().any()] 
# mean_data_by_site_id = test_dataset.groupby('site_id')[missing_cols].transform('mean')
# test_dataset.fillna(mean_data_by_site_id, inplace=True)

# #Add hour, time of year, and weekend columns
# test_dataset['hour'] = test_dataset['timestamp'].dt.hour
# test_dataset['day_of_year'] = (test_dataset['timestamp'] - pd.Timestamp('2016-01-01')).dt.days%365
# test_dataset['is_weekend'] = test_dataset['timestamp'].dt.weekday.isin([5,6]).astype(int)

In [10]:
# pickle.dump( test_dataset, open( "test_dataset.p", "wb" ) )
# del test_dataset
gc.collect()

28

# 5. Reduce Memory Usage

In [11]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [12]:
train_dataset = pickle.load( open( "train_dataset.p", "rb" ) )
# test_dataset = pickle.load( open( "test_dataset.p", "rb" ) )

In [13]:
train_dataset = reduce_mem_usage(train_dataset)
# test_dataset = reduce_mem_usage(test_dataset)

Mem. usage decreased to 393.30 Mb (67.3% reduction)
Mem. usage decreased to 811.22 Mb (67.3% reduction)


# 6. Further clean dataset to handle outlier data and to prepare to train model

In [14]:
#We would like to rescale the meter reading column for each building and meter reading to prevent outliers from skewing the results.
#This is a class to achieve that for any chosen groups. It is a modified version of code by Szymon Maszke: 
#https://stackoverflow.com/questions/55601928/apply-multiple-standardscalers-to-individual-groups

from sklearn.base import clone
class GroupTargetTransform:
    def __init__(self, transformation):
        self.transformation = transformation
        self._group_transforms = {} #this library will hold the group transforms

    def _call_with_function(self, X, y, function: str):
        yhat = pd.Series(dtype = 'float32')#this will hold the rescaled target data
        X['target'] = pd.Series(y, index=X.index)
        for gr in X.groupby(self.features):
            n = gr[0] #this is a tuple id for the group
            g_X = gr[1] #this is the group dataframe
            g_yhat = getattr(self._group_transforms[n], function)(g_X['target'].values.reshape(-1,1))#scale the target variable
            g_yhat = pd.Series(g_yhat.flatten(), index = g_X.index)
            yhat = yhat.append(g_yhat)
        X.drop('target', axis=1, inplace = True)
        return yhat.sort_index()
    
    def fit(self, X, y, features):
        self.features = features
        X['target'] = pd.Series(y, index=X.index) 
        for gr in X.groupby(self.features):
            n = gr[0] #this is a tuple id for the group
            g_X = gr[1] #this is the group dataframe
            sc = clone(self.transformation) #create a new instance of the transform
            self._group_transforms[n] = sc.fit(g_X['target'].values.reshape(-1,1))
        X.drop('target', axis=1, inplace=True)
        return self

    def transform(self, X, y):
        return self._call_with_function(X, y, "transform")

    def fit_transform(self, X, y, features):
        self.fit(X, y, features)
        return self.transform(X, y)

    def inverse_transform(self, X, y):
        return self._call_with_function(X, y, "inverse_transform")

In [15]:
#rescale the target variable for each building and meter type.
from sklearn.preprocessing import MinMaxScaler

scaler = GroupTargetTransform(MinMaxScaler(feature_range = (0,2000))) #2000 is roughly the average meter reading for all the train data
train_dataset['meter_reading'] = scaler.fit_transform(train_dataset, train_dataset['meter_reading'], ['building_id', 'meter'])
# convert to log(y+1) so the RMSE evaluation metric is actually giving the RMSLE
train_dataset['meter_reading'] = np.log1p(train_dataset['meter_reading']) 

In [16]:
Log("file.log", formatLOG , "Cleaning dataset Successfully.", logging.INFO)

# 7. Train Model

In [17]:
Log("file.log", formatLOG , "Training Model.", logging.INFO)

In [18]:
# pickle.dump( test_dataset, open( "test_dataset.p", "wb" ) )
pickle.dump( train_dataset, open( "train_dataset.p", "wb" ) )
#del test_dataset

In [19]:
%%time
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error

#Drop timestamp
train_dataset = train_dataset.drop("timestamp", axis = 1)
# test_dataset = test_dataset.drop("timestamp", axis = 1)

Wall time: 1.56 s


In [20]:
#prepare training data
X = train_dataset

In [21]:
#prepare training data
X = train_dataset
X = train_dataset.dropna(subset=['meter_reading']) #drop all rows where the meter reading is not included
y = train_dataset["meter_reading"]

#Remove meter_reading so that it does not have the "answers"
del X['meter_reading']
del X['site_id']

In [22]:
# print("Shape train: %s, test: %s" % (train_dataset.shape, test_dataset.shape))

Shape train: (8086440, 18), test: (16679040, 18)


In [23]:
# 80% train, 20% eval
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = 0.2, random_state=13)

In [24]:
# using the 80% train, I take out 20% for evaluation of accuracy
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, random_state=12)

In [25]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'learning_rate': 0.005,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "n_estimators": 1000
}

In [26]:
model = lgb.LGBMRegressor(**params)

In [27]:
model.fit(X_train, y_train,
        eval_set=[(X_eval, y_eval)],
        eval_metric='l1',
        early_stopping_rounds=1000)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.421638 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM

[143]	valid_0's l1: 1.23831	valid_0's l2: 2.9869
[144]	valid_0's l1: 1.2371	valid_0's l2: 2.98183
[145]	valid_0's l1: 1.23573	valid_0's l2: 2.97594
[146]	valid_0's l1: 1.23434	valid_0's l2: 2.97
[147]	valid_0's l1: 1.23287	valid_0's l2: 2.96371
[148]	valid_0's l1: 1.23212	valid_0's l2: 2.96087
[149]	valid_0's l1: 1.23074	valid_0's l2: 2.95508
[150]	valid_0's l1: 1.22931	valid_0's l2: 2.9491
[151]	valid_0's l1: 1.22796	valid_0's l2: 2.94348
[152]	valid_0's l1: 1.22675	valid_0's l2: 2.93835
[153]	valid_0's l1: 1.22503	valid_0's l2: 2.93111
[154]	valid_0's l1: 1.22371	valid_0's l2: 2.92564
[155]	valid_0's l1: 1.22294	valid_0's l2: 2.92271
[156]	valid_0's l1: 1.22161	valid_0's l2: 2.91725
[157]	valid_0's l1: 1.22105	valid_0's l2: 2.91518
[158]	valid_0's l1: 1.21967	valid_0's l2: 2.90947
[159]	valid_0's l1: 1.21827	valid_0's l2: 2.90366
[160]	valid_0's l1: 1.21752	valid_0's l2: 2.90089
[161]	valid_0's l1: 1.21609	valid_0's l2: 2.8949
[162]	valid_0's l1: 1.21443	valid_0's l2: 2.88802
[163]	v

[308]	valid_0's l1: 1.07556	valid_0's l2: 2.36739
[309]	valid_0's l1: 1.07475	valid_0's l2: 2.36461
[310]	valid_0's l1: 1.07381	valid_0's l2: 2.36122
[311]	valid_0's l1: 1.0731	valid_0's l2: 2.3589
[312]	valid_0's l1: 1.07246	valid_0's l2: 2.35675
[313]	valid_0's l1: 1.07209	valid_0's l2: 2.35573
[314]	valid_0's l1: 1.07138	valid_0's l2: 2.35334
[315]	valid_0's l1: 1.07082	valid_0's l2: 2.35144
[316]	valid_0's l1: 1.07005	valid_0's l2: 2.34892
[317]	valid_0's l1: 1.06969	valid_0's l2: 2.34793
[318]	valid_0's l1: 1.06898	valid_0's l2: 2.34555
[319]	valid_0's l1: 1.06862	valid_0's l2: 2.34452
[320]	valid_0's l1: 1.06798	valid_0's l2: 2.34226
[321]	valid_0's l1: 1.06732	valid_0's l2: 2.33996
[322]	valid_0's l1: 1.06658	valid_0's l2: 2.33747
[323]	valid_0's l1: 1.06627	valid_0's l2: 2.3366
[324]	valid_0's l1: 1.06551	valid_0's l2: 2.33403
[325]	valid_0's l1: 1.06484	valid_0's l2: 2.33176
[326]	valid_0's l1: 1.06407	valid_0's l2: 2.32916
[327]	valid_0's l1: 1.06335	valid_0's l2: 2.32675
[32

[473]	valid_0's l1: 0.990986	valid_0's l2: 2.09964
[474]	valid_0's l1: 0.990434	valid_0's l2: 2.09785
[475]	valid_0's l1: 0.990065	valid_0's l2: 2.09666
[476]	valid_0's l1: 0.989573	valid_0's l2: 2.09513
[477]	valid_0's l1: 0.989369	valid_0's l2: 2.0946
[478]	valid_0's l1: 0.988953	valid_0's l2: 2.09344
[479]	valid_0's l1: 0.988786	valid_0's l2: 2.09292
[480]	valid_0's l1: 0.988463	valid_0's l2: 2.09187
[481]	valid_0's l1: 0.988007	valid_0's l2: 2.09049
[482]	valid_0's l1: 0.987657	valid_0's l2: 2.08931
[483]	valid_0's l1: 0.987283	valid_0's l2: 2.08823
[484]	valid_0's l1: 0.986804	valid_0's l2: 2.08681
[485]	valid_0's l1: 0.986356	valid_0's l2: 2.08544
[486]	valid_0's l1: 0.985962	valid_0's l2: 2.08427
[487]	valid_0's l1: 0.985519	valid_0's l2: 2.08295
[488]	valid_0's l1: 0.985367	valid_0's l2: 2.08252
[489]	valid_0's l1: 0.985081	valid_0's l2: 2.08166
[490]	valid_0's l1: 0.984648	valid_0's l2: 2.08034
[491]	valid_0's l1: 0.984479	valid_0's l2: 2.0798
[492]	valid_0's l1: 0.984137	vali

[635]	valid_0's l1: 0.94107	valid_0's l2: 1.95417
[636]	valid_0's l1: 0.940836	valid_0's l2: 1.95342
[637]	valid_0's l1: 0.940519	valid_0's l2: 1.9526
[638]	valid_0's l1: 0.940177	valid_0's l2: 1.95165
[639]	valid_0's l1: 0.939832	valid_0's l2: 1.95062
[640]	valid_0's l1: 0.939509	valid_0's l2: 1.94962
[641]	valid_0's l1: 0.939304	valid_0's l2: 1.94908
[642]	valid_0's l1: 0.939045	valid_0's l2: 1.94842
[643]	valid_0's l1: 0.938791	valid_0's l2: 1.9478
[644]	valid_0's l1: 0.938676	valid_0's l2: 1.94749
[645]	valid_0's l1: 0.938461	valid_0's l2: 1.94692
[646]	valid_0's l1: 0.938195	valid_0's l2: 1.94623
[647]	valid_0's l1: 0.937993	valid_0's l2: 1.94569
[648]	valid_0's l1: 0.93774	valid_0's l2: 1.94509
[649]	valid_0's l1: 0.937461	valid_0's l2: 1.94432
[650]	valid_0's l1: 0.937211	valid_0's l2: 1.9437
[651]	valid_0's l1: 0.937119	valid_0's l2: 1.94345
[652]	valid_0's l1: 0.936749	valid_0's l2: 1.94228
[653]	valid_0's l1: 0.936649	valid_0's l2: 1.94198
[654]	valid_0's l1: 0.936524	valid_0

[797]	valid_0's l1: 0.905367	valid_0's l2: 1.84463
[798]	valid_0's l1: 0.905189	valid_0's l2: 1.84405
[799]	valid_0's l1: 0.90496	valid_0's l2: 1.84332
[800]	valid_0's l1: 0.90473	valid_0's l2: 1.84256
[801]	valid_0's l1: 0.904545	valid_0's l2: 1.84196
[802]	valid_0's l1: 0.904341	valid_0's l2: 1.84129
[803]	valid_0's l1: 0.904127	valid_0's l2: 1.84059
[804]	valid_0's l1: 0.903995	valid_0's l2: 1.84021
[805]	valid_0's l1: 0.903802	valid_0's l2: 1.8396
[806]	valid_0's l1: 0.903627	valid_0's l2: 1.83906
[807]	valid_0's l1: 0.903433	valid_0's l2: 1.83847
[808]	valid_0's l1: 0.903317	valid_0's l2: 1.83799
[809]	valid_0's l1: 0.903112	valid_0's l2: 1.83731
[810]	valid_0's l1: 0.902877	valid_0's l2: 1.83649
[811]	valid_0's l1: 0.902639	valid_0's l2: 1.83587
[812]	valid_0's l1: 0.90244	valid_0's l2: 1.83531
[813]	valid_0's l1: 0.90231	valid_0's l2: 1.83491
[814]	valid_0's l1: 0.902	valid_0's l2: 1.83384
[815]	valid_0's l1: 0.901829	valid_0's l2: 1.83329
[816]	valid_0's l1: 0.901671	valid_0's 

[959]	valid_0's l1: 0.880856	valid_0's l2: 1.77123
[960]	valid_0's l1: 0.880615	valid_0's l2: 1.77033
[961]	valid_0's l1: 0.880459	valid_0's l2: 1.76996
[962]	valid_0's l1: 0.880309	valid_0's l2: 1.76961
[963]	valid_0's l1: 0.88016	valid_0's l2: 1.76925
[964]	valid_0's l1: 0.880036	valid_0's l2: 1.76892
[965]	valid_0's l1: 0.87995	valid_0's l2: 1.76865
[966]	valid_0's l1: 0.879797	valid_0's l2: 1.76809
[967]	valid_0's l1: 0.879706	valid_0's l2: 1.76785
[968]	valid_0's l1: 0.879621	valid_0's l2: 1.76763
[969]	valid_0's l1: 0.879471	valid_0's l2: 1.76727
[970]	valid_0's l1: 0.879286	valid_0's l2: 1.76653
[971]	valid_0's l1: 0.879181	valid_0's l2: 1.76628
[972]	valid_0's l1: 0.879091	valid_0's l2: 1.76602
[973]	valid_0's l1: 0.878959	valid_0's l2: 1.76562
[974]	valid_0's l1: 0.87885	valid_0's l2: 1.76537
[975]	valid_0's l1: 0.878735	valid_0's l2: 1.76492
[976]	valid_0's l1: 0.8786	valid_0's l2: 1.76447
[977]	valid_0's l1: 0.878484	valid_0's l2: 1.76408
[978]	valid_0's l1: 0.878359	valid_0

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.8,
              learning_rate=0.005, max_bin=512, max_depth=8, n_estimators=1000,
              num_leaves=128, objective='regression', task='train', verbose=0)

In [28]:
def clip(x):
    return np.clip(x, a_min=0, a_max=None)

In [29]:
y_pred = clip(model.predict(X_test, num_iteration=model.best_iteration_))

In [30]:
# Basic RMSE
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_test) ** 0.5, 5))

The rmse of prediction is: 0.43116


In [31]:
# save the model to disk
#model.booster_.save_model('Data/Model/LGBM_Model.txt')

filename = 'Data/Model/LGBM_Model.sav'
pickle.dump(model, open(filename, 'wb'))